In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fmnist = keras.datasets.fashion_mnist

In [4]:
(x_train, y_train), (x_test, y_test) = fmnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
#scaling the images

x_train = x_train/255.0
x_test = x_test/255.0

In [6]:
x_train.reshape(len(x_train),28,28,1)
x_test=x_test.reshape(len(x_test),28,28,1)

In [7]:
import kerastuner


<ipython-input-7-f829eee6c585>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner


**We can build a function that builds a model for us for a given set of hyperparameters. The function uses an arguement called 'hp'. hp can be used to define the ranges and choices for our hyperparameters. Use hp.Int when you want
 the model to try from a range of values and hp.Choice when you want the model to try for a specific set of values.**

In [11]:

def build_model(hp):

  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv1_filters',min_value=32,max_value=128,step=16),
      kernel_size = hp.Choice('conv1_kernel_size',values=[3,5]),
      activation = 'relu',
      input_shape = (28,28,1)),

      keras.layers.Conv2D(
          filters = hp.Int('conv2_filters',min_value=32,max_value=64,step=16),
          kernel_size = hp.Choice('conv2_kernel_size',values=[3,5]),
          activation = 'relu'

      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense1_units',min_value=32,max_value=128,step=16),
          activation = 'relu'
      ),
      keras.layers.Dense(10,activation='softmax')
      ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                                                  loss='sparse_categorical_crossentropy',
                                                  metrics=['accuracy'])


  return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

**Initiate a tuner, I am using Randomsearch ( refer the docs for keras tuner for more options). Specify the 'objective' (what do you wish to maximise or minimise), 'max_trials' (how many times do you want the tuner to try different models) and other keyword arguements.**

In [13]:
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     directory='output',
                     project_name='MNISTFASHION_TUNED')

**The .search() method runs the trials for the number of times you specify. You can access the best model using the get_best_models(). Train and predict using the best model.**

In [14]:
tuner.search(x_train,y_train,validation_split=0.1,epochs=3)

Trial 5 Complete [00h 00m 27s]
val_accuracy: 0.9045000076293945

Best val_accuracy So Far: 0.9075000286102295
Total elapsed time: 00h 03m 07s


In [23]:
best_model = tuner.get_best_models()[0]

In [24]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 112)       2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 48)        134448    
                                                                 
 flatten (Flatten)           (None, 19200)             0         
                                                                 
 dense (Dense)               (None, 96)                1843296   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 1981626 (7.56 MB)
Trainable params: 1981626 (7.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
best_model.fit(x_train,y_train,epochs=10,validation_split=0.3,initial_epoch=3)

Epoch 4/10
1313/1313 [==============================] - 10s 7ms/step - loss: 0.1684 - accuracy: 0.9356 - val_loss: 0.1923 - val_accuracy: 0.9281
Epoch 5/10
1313/1313 [==============================] - 9s 7ms/step - loss: 0.1293 - accuracy: 0.9516 - val_loss: 0.2106 - val_accuracy: 0.9273
Epoch 6/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0995 - accuracy: 0.9626 - val_loss: 0.2371 - val_accuracy: 0.9297
Epoch 7/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0777 - accuracy: 0.9711 - val_loss: 0.2564 - val_accuracy: 0.9237
Epoch 8/10
1313/1313 [==============================] - 9s 7ms/step - loss: 0.0572 - accuracy: 0.9784 - val_loss: 0.3276 - val_accuracy: 0.9196
Epoch 9/10
1313/1313 [==============================] - 8s 6ms/step - loss: 0.0481 - accuracy: 0.9820 - val_loss: 0.3321 - val_accuracy: 0.9218
Epoch 10/10
1313/1313 [==============================] - 9s 7ms/step - loss: 0.0370 - accuracy: 0.9869 - val_loss: 0.3830 - val_accurac

In [39]:
single_predictions = []
for pred in predictions:

  final = np.argmax(pred)
  single_predictions.append(final)



In [41]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [43]:
score = accuracy_score(y_test,single_predictions)

In [45]:
score*100

90.14

In [46]:
cm = confusion_matrix(y_true=y_test,y_pred=single_predictions)

In [47]:
cm

array([[838,   9,  16,  19,   3,   2, 104,   1,   8,   0],
       [  1, 989,   0,   7,   0,   0,   2,   0,   1,   0],
       [ 16,   2, 798,  11,  84,   0,  88,   0,   1,   0],
       [ 15,  17,  10, 907,  25,   0,  26,   0,   0,   0],
       [  1,   3,  40,  33, 867,   0,  54,   0,   2,   0],
       [  0,   0,   0,   0,   0, 979,   0,  11,   1,   9],
       [106,   5,  57,  21,  77,   0, 727,   0,   7,   0],
       [  0,   1,   0,   0,   0,  10,   0, 965,   0,  24],
       [  3,   0,   2,   4,   4,   3,   5,   2, 976,   1],
       [  0,   0,   1,   0,   0,   5,   1,  25,   0, 968]])

In [51]:
best_model.sav

Results summary
Results in output/MNISTFASHION_TUNED
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
conv1_filters: 112
conv1_kernel_size: 5
conv2_filters: 48
conv2_kernel_size: 5
dense1_units: 96
learning_rate: 0.001
Score: 0.9075000286102295

Trial 0 summary
Hyperparameters:
conv1_filters: 112
conv1_kernel_size: 3
conv2_filters: 64
conv2_kernel_size: 5
dense1_units: 96
learning_rate: 0.001
Score: 0.9070000052452087

Trial 4 summary
Hyperparameters:
conv1_filters: 64
conv1_kernel_size: 5
conv2_filters: 48
conv2_kernel_size: 5
dense1_units: 48
learning_rate: 0.001
Score: 0.9045000076293945

Trial 3 summary
Hyperparameters:
conv1_filters: 32
conv1_kernel_size: 5
conv2_filters: 32
conv2_kernel_size: 3
dense1_units: 64
learning_rate: 0.01
Score: 0.859333336353302

Trial 2 summary
Hyperparameters:
conv1_filters: 112
conv1_kernel_size: 3
conv2_filters: 48
conv2_kernel_size: 3
dense1_units: 112
learning_rate: 0.01
Score: 0.857333362102

In [53]:
best_model.save('/content/output')